In [1]:
import requests
import xml.etree.cElementTree as et
import json
import pandas as pd
import numpy as np

In [2]:
from tqdm import tqdm_notebook as tqdmn

In [3]:
from io import StringIO

In [4]:
# ids = [5219, 5247, 5248]
# ids = [4981, 4982, 4983, 4984, 4985]
ids = [5965, 5966, 5967]

In [5]:
flags_url = 'http://rating.chgk.info/tournaments.php?tournament_id={}&download_data=tournament_export_teams_with_flags'

In [6]:
results_pd = [pd.read_csv(StringIO(requests.get(flags_url.format(i)).text), skiprows=1, index_col=None,
                          sep=',', error_bad_lines=False, 
                          names=['idteam','name_{}'.format(i), 'city_{}'.format(i),'place_{}'.format(i),'questions_{}'.format(i),'flags_{}'.format(i)]) for i in tqdmn(ids)]

In [7]:
results_json = [requests.get('http://rating.chgk.info/api/tournaments/{}/list'.format(id)).text for id in tqdmn(ids)]
results_pd = [pd.read_json(res) for res in results_json]

OverflowError: int too big to convert

In [7]:
results_pd[0].head()

,idteam,name_5965,city_5965,place_5965,questions_5965,flags_5965
0,48871,Красные рассветы,Уфа,"1,5",26.0,NaN
1,60574,Пернатонога Кукумявка,Киев,"1,5",26.0,NaN
2,60990,Шузы из Медузы,Киев,3,25.0,NaN
3,50934,"Те, кого нельзя называть",Киев,"5,5",24.0,NaN
4,52175,Сборная Вышки,Москва,"5,5",24.0,Вуз


In [8]:
from functools import reduce

In [9]:
merged_df =  reduce(lambda l, r: pd.merge(l, r, on=['idteam'], how='outer'), results_pd)

In [10]:
merged_df.head()

,idteam,name_5965,city_5965,place_5965,questions_5965,flags_5965,name_5966,city_5966,place_5966,questions_5966,flags_5966,name_5967,city_5967,place_5967,questions_5967,flags_5967
0,48871,Красные рассветы,Уфа,"1,5",26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60574,Пернатонога Кукумявка,Киев,"1,5",26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60990,Шузы из Медузы,Киев,3,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50934,"Те, кого нельзя называть",Киев,"5,5",24.0,NaN,NaN,NaN,NaN,NaN,NaN,"Те, кого нельзя называть",Киев,"5,5",31.0,NaN
4,52175,Сборная Вышки,Москва,"5,5",24.0,Вуз,Сборная Вышки,Москва,12,22.0,Вуз,Сборная Вышки,Москва,17,28.0,! Вуз


In [11]:
len(merged_df)

251

In [12]:
merged_df['name'] = [' / '.join(set(r.dropna().tolist())) for i, r in merged_df.loc[:,merged_df.columns.str.startswith('name_')].iterrows()]
merged_df = merged_df.drop(merged_df.columns[merged_df.columns.str.startswith('name_')], axis=1)
merged_df.head()

,idteam,city_5965,place_5965,questions_5965,flags_5965,city_5966,place_5966,questions_5966,flags_5966,city_5967,place_5967,questions_5967,flags_5967,name
0,48871,Уфа,"1,5",26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Красные рассветы
1,60574,Киев,"1,5",26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пернатонога Кукумявка
2,60990,Киев,3,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Шузы из Медузы
3,50934,Киев,"5,5",24.0,NaN,NaN,NaN,NaN,NaN,Киев,"5,5",31.0,NaN,"Те, кого нельзя называть"
4,52175,Москва,"5,5",24.0,Вуз,Москва,12,22.0,Вуз,Москва,17,28.0,! Вуз,Сборная Вышки


Chech whether all flags and cities are equal, merge them

In [13]:
for param in ['flags', 'city']:
    param_cols = merged_df.loc[:,merged_df.columns.str.startswith(param+'_')]
    param_join = param_cols.apply(lambda x: set([e for c in x.dropna() if not pd.isna(c) for e in set(c.split())]), axis=1)
    same = param_join.apply(lambda x: len(x) < 2)
    param_join = param_join.apply(lambda x: ' / '.join(x))
    
    if not np.all(same):
        diff_ids = merged_df.loc[np.logical_not(same), merged_df.columns.str.contains('(idteam)|(name)|({}_)'.format(param))]
#         print('These teams had different {}:'.format(param))
#         print(diff_ids)
        
    merged_df = merged_df.drop(merged_df.columns[merged_df.columns.str.startswith(param)], axis=1)
    merged_df[param] = param_join

C:\Anaconda3\envs\pommenv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [14]:
merged_df['questions'] = np.nansum(merged_df.loc[:,merged_df.columns.str.startswith('questions_')], axis=1)
merged_df['place'] = np.nansum(np.array(merged_df.loc[:,merged_df.columns.str.startswith('place_')].apply(lambda x: x.str.replace(',','.')), dtype=np.float), axis=1)
merged_df.sort_values(['questions','place'], inplace=True, ascending=[False,True])
merged_df.replace(np.nan, 0, inplace=True)

Remove worst tour

In [15]:
np.nanmin(merged_df.loc[:,merged_df.columns.str.startswith('questions_')], axis=1)

array([19., 22., 22., 19., 20., 22., 20., 20., 14., 16., 20., 21., 17.,
       20., 17., 17., 19., 14., 17., 15., 18., 16., 17., 15., 17., 13.,
       15.,  0., 13., 17., 15., 15., 11.,  0., 12., 13.,  9., 14.,  0.,
        0., 10., 12., 12., 14.,  9.,  8.,  0.,  0., 13., 13., 13., 11.,
        0., 12., 11.,  0.,  0.,  0.,  0.,  8.,  6.,  8.,  0.,  9., 10.,
       10.,  9.,  0.,  0.,  0., 10.,  9.,  0.,  8.,  7.,  0.,  0.,  0.,
        0.,  8.,  6.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  3.,  5.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  4.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  5.,  0.,
        0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [16]:
merged_df['wo_worst_questions'] = merged_df.questions - np.nanmin(merged_df.loc[:,merged_df.columns.str.startswith('questions_')], axis=1)
# merged_df.sort_values(['wo_worst_questions','questions'], inplace=True, ascending=[False,False])
merged_df.sort_values(['questions', 'place'], inplace=True, ascending=[False,False])
merged_df = merged_df[merged_df.columns.tolist()[:1] + merged_df.columns.tolist()[-6:] + merged_df.columns.tolist()[1:-6]]

In [17]:
merged_df.columns.tolist()[:1] + merged_df.columns.tolist()[:1]

['idteam', 'idteam']

In [23]:
merged_df.flags = merged_df.flags.str.replace(' / !','').replace('!','')

In [24]:
merged_df.head(20)

,idteam,name,flags,city,questions,place,wo_worst_questions,place_5965,questions_5965,place_5966,questions_5966,place_5967,questions_5967
27,73429,Московский ордена Ленина,Вуз,Москва,76.0,45.5,57.0,27,19.0,"1,5",29.0,17,28.0
10,8424,Виталия,,Калининград,75.0,37.0,53.0,13,22.0,3,26.0,21,27.0
4,52175,Сборная Вышки,Вуз,Москва,74.0,34.5,52.0,"5,5",24.0,12,22.0,17,28.0
17,69251,Пифийский перформанс,Вуз,сборная,73.0,51.5,54.0,17,21.0,"31,5",19.0,3,33.0
6,66459,Совушки,Вуз,Минск,71.0,50.0,51.0,"5,5",24.0,"23,5",20.0,21,27.0
9,73804,Капитан ненавидит море,,Казань,69.0,59.0,47.0,9,23.0,12,22.0,38,24.0
77,59696,Лебеди облюбовали поле рядом с шоссе Валга-Уул...,Вуз,Тарту,68.0,92.0,54.0,"77,5",14.0,5,24.0,"9,5",30.0
11,54075,Реситуасексулузон,Вуз,Москва,68.0,61.5,48.0,13,22.0,"23,5",20.0,25,26.0
21,72697,Референц-грушоид,Вуз,Москва,68.0,59.5,48.0,21,20.0,"8,5",23.0,30,25.0
15,60143,Батяня Шаббат,Вуз,Москва,67.0,82.5,51.0,17,21.0,56,16.0,"9,5",30.0


In [25]:
merged_df.to_csv('../data/ochvr_2019_3.csv', index=False, encoding='utf-8-sig')